In [2]:
#Needleman-Wunsch algorithm을 사용한 MSA 구현
import time
import numpy as np
    
MINUSINF=-999999 #아직 점수 계산이 되지 않는 영역에 설정하는 점수
#match, gap에 매기는 점수
MATCH=1
GAP=-1
sequenceList=[] #sequence list
sequenceLengthList=[] #sequence의 길이 list
scoreMatrix=[] #score matrix
scoreMatrixLength=1 #score matrix의 길이
divisorList=[] #행렬<->인덱스 변환에 필요한 정보


#행렬<->인덱스 변환에 필요한 정보 설정
def initializeInfo():
    divisor=1
    dividend=1
    remainder=1
    for i in range(len(sequenceLengthList)-1,-1,-1):
        divisor=1
        for j in range(0,i):
            divisor*=sequenceLengthList[len(sequenceLengthList)-j-1]
        divisorList.append(divisor)

#인덱스->행렬 변환
def listToMatrix(inputList):
    returnIndex=[]
    dividend=1
    quotient=inputList
    for i in range(len(sequenceLengthList)-1,-1,-1):
        dividend, quotient=divmod(quotient,divisorList[len(sequenceLengthList)-i-1])
        returnIndex.append(dividend)
    return tuple(returnIndex)

         
#행렬->인덱스 변환
def matrixToList(inputMatrix):
    inputMatrix=list(inputMatrix)
    sum=0
    for i in range(0,len(sequenceLengthList)):
        sum+=divisorList[i]*inputMatrix[i]
    return sum



#프로그램 시작, sequence 입력
print("sequence를 입력하세요 : ", end='')
temp=input().split(',')
for i in temp:
    sequenceList.append(i)
    sequenceLengthList.append(len(i)+1)
    scoreMatrixLength*=len(i)+1

print("")
start = time.time()    
#score
for i in range(0,scoreMatrixLength):
    scoreMatrix.append(i)

scoreMatrix=np.array(scoreMatrix)
scoreMatrix=scoreMatrix.reshape(sequenceLengthList)


#score 초기값 설정 (제일 가장자리)
for i in range(0,len(sequenceLengthList)):
    for j in range(0,sequenceLengthList[len(sequenceLengthList)-i-1]):
        addNum=1
        for k in range(0,i):
            addNum*=sequenceLengthList[len(sequenceLengthList)-k-1]
        scoreMatrix=np.where(scoreMatrix==j*addNum,GAP*j,scoreMatrix)
scoreMatrix=np.where(scoreMatrix>0,MINUSINF,scoreMatrix)


#1무터 2^n-1까지의 이진수 String을 갖는 list 생성
nearList=[]
for i in range(1,2**len(sequenceLengthList)):
    temp=format(i, 'b')
    while(len(temp)<len(sequenceLengthList)):
        temp="0"+temp
    nearList.append(temp)
    
initializeInfo()


#점수 설정
for i in range(0,scoreMatrixLength):
    if(i%100000==0):
        temp=i*100/scoreMatrixLength
        print(temp,"% complete")
    target=listToMatrix(i)
    targetNearList=[]
    for j in nearList:
    #근처 좌표를 설정한다.
        near=()
        for k in range(0,len(target)):
            if(j[k]=='0'):
                near+=(target[k],)
            elif(j[k]=='1'):
                near+=(target[k]-1,)
        if(not(-1 in near)):
            targetNearList.append(near)
        
    
    
    #설정되지 않은 좌표가 있는 경우 점수를 설정하지 않고 넘어간다.
    flag=True
    for j in targetNearList:
        if(scoreMatrix[j]==MINUSINF):
            flag=False
     
    #근처 좌표의 점수가 모두 설정된 경우, 점수를 설정한다.
    if((flag)and(scoreMatrix[target]==MINUSINF)):
        max=MINUSINF
        for j in targetNearList:
            flag=True
            for k in range(0,len(j)):
                if(target[k]==j[k]):
                    flag=False
            
            #대각선 방향에서 온 경우
            if(flag):
                seq=sequenceList[0][j[0]]
                flag2=True
                for k in range(0,len(j)):
                    if(seq!=sequenceList[k][j[k]]):
                        flag2=False
                        
                #Match
                if(flag2==True):
                    max=scoreMatrix[j]+MATCH if(scoreMatrix[j]+MATCH>max) else max
                else:
                    max=scoreMatrix[j]+GAP if(scoreMatrix[j]+GAP>max) else max
            else:
                max=scoreMatrix[j]+GAP if(scoreMatrix[j]+GAP>max) else max
                
        scoreMatrix[target]=max
        
        
    
    
    
#backtracking
posNow=listToMatrix(scoreMatrixLength-1)
path=[posNow]
#debug

while(True):
    #posNow의 근처 좌표를 설정한다.
    targetNearList=[]
    for i in nearList:
        near=()
        for j in range(0,len(posNow)):
            if(i[j]=='0'):
                near+=(posNow[j],)
            elif(i[j]=='1'):
                near+=(posNow[j]-1,)
        if(not(-1 in near)):
            targetNearList.append(near)
            
    #갈 방향을 정하고, 원점으로 돌아왔다면 backtracking을 종료한다.        
    for i in targetNearList:
        dest=()
        flag=True
        for j in range(0,len(i)):
            if(int(i[j])==posNow[j]):
                flag=False
            dest+=(posNow[j]-int(i[j]),)
                        
        if(flag):
            seq=sequenceList[0][i[0]]
            flag2=True
            for j in range(0,len(i)):
                if(seq!=sequenceList[j][i[j]]):
                    flag2=False
            #Match
            if(flag2):
                if(scoreMatrix[i]+MATCH==scoreMatrix[posNow]):
                    posNow=i
                    path.append(i)
                    break
                else:
                    continue
            #Mismatch
            else:
                if(scoreMatrix[i]+GAP==scoreMatrix[posNow]):
                    posNow=i
                    path.append(i)
                    break
                else:
                    continue
        else:
            if(scoreMatrix[i]+GAP==scoreMatrix[posNow]):
                print("mismatch")
                posNow=i
                path.append(i)
                break
            else:
                continue         
            
        
            
    
    #시작 지점으로 돌아오면 종료
    flag=True
    for i in posNow:
        if(i!=0):
            flag=False
    if(flag==True):
        break
        

path.reverse()

#debug
print("")
print("Path")
print(path)

#방향 저장
path2=[]

for i in range(0,len(path)-1):
    temp=[]
    for j in range(0,len(path[i])):
        temp.append(path[i+1][j]-path[i][j])
    path2.append(temp)

    

#backtracking한 결과를 출력한다.
resultSeq=[]

for i in range(0,len(sequenceList)):
    resultSeq.append([])

for i in range(0,len(path2)):
    for j in range(0,len(sequenceList)):
        if(path2[i][j]==1):
            resultSeq[j].append(sequenceList[j][0])
            sequenceList[j]=sequenceList[j][1:len(sequenceList[j])]
        elif(path2[i][j]==0):
            resultSeq[j].append('-')
        else:
            print("bug")

#MSA 결과 출력
print("")
print("result")
for i in resultSeq:
    for j in i:
        print(j, end=' ')
    print("")

print("")
print("score : ",scoreMatrix[listToMatrix(scoreMatrixLength-1)])
print("")
print("execution time :", time.time() - start,"sec")
        
    

    



sequence를 입력하세요 : CTGATCATCGACCGGGATGA,CTGCTCATCGAGAGGGATCA,CTGCTCATCGAGCCGCATGA,CTGCGCATCGAGCGGGTTGA

0.0 % complete
51.41890467449262 % complete
mismatch
mismatch
mismatch
mismatch

Path
[(0, 0, 0, 0), (1, 1, 1, 1), (2, 2, 2, 2), (3, 3, 3, 3), (4, 4, 4, 4), (5, 5, 5, 5), (6, 6, 6, 6), (7, 7, 7, 7), (8, 8, 8, 8), (9, 9, 9, 9), (10, 10, 10, 10), (11, 11, 11, 11), (12, 12, 11, 12), (13, 13, 11, 13), (14, 14, 12, 14), (15, 15, 13, 15), (15, 15, 14, 15), (16, 16, 15, 16), (17, 17, 16, 17), (17, 17, 17, 17), (18, 18, 18, 18), (19, 19, 19, 19), (20, 20, 20, 20)]

result
C T G A T C A T C G A C C G G - G A - T G A 
C T G C T C A T C G A G A G G - G A - T C A 
C T G C T C A T C G A - - G C C G C A T G A 
C T G C G C A T C G A G C G G - G T - T G A 

score :  4

execution time : 16.38126516342163 sec
